In [59]:
import os 
import cv2
import numpy as np 
from tqdm import tqdm
import pandas as pd
import glob
from spiral_tools.utils import path_sequence,show_images_figure
from IPython.display import clear_output
from spiral_tools.utils import get_screen_resolution

In [60]:
vid_file_path = path_sequence(".","stabilized_bidirection_no_vehicle_5v5_01.mp4")
lbls_file_path = path_sequence(".","bidirection_no_vehicle_5v5_01")
pix2meter_file_path = path_sequence(lbls_file_path,"ratio_pixel2meter_ground.txt")

save_frames_path = path_sequence(".","fetched_frames")
save_labels_path = path_sequence(".","fetched_labels")

In [61]:
#  oranı okuyalım
f = open(pix2meter_file_path)
ratio =  float(f.readlines()[0])
f.close()
print(ratio)

# obje labelları
objects_labels_files_list = glob.glob(path_sequence(lbls_file_path,"p*.csv"))
print(objects_labels_files_list)


45.032
['./bidirection_no_vehicle_5v5_01\\p1.csv', './bidirection_no_vehicle_5v5_01\\p10.csv', './bidirection_no_vehicle_5v5_01\\p2.csv', './bidirection_no_vehicle_5v5_01\\p3.csv', './bidirection_no_vehicle_5v5_01\\p4.csv', './bidirection_no_vehicle_5v5_01\\p5.csv', './bidirection_no_vehicle_5v5_01\\p6.csv', './bidirection_no_vehicle_5v5_01\\p7.csv', './bidirection_no_vehicle_5v5_01\\p8.csv', './bidirection_no_vehicle_5v5_01\\p9.csv']


In [62]:
def get_labels(label_file_path_list):
    frames = {}
    
    for file_path in tqdm(objects_labels_files_list):
        clear_output()        
        print("işleniyor. ",file_path)
        df = pd.read_csv(file_path)
        
        for idx in range(len(df)):
            frame_idx = int(df.iloc[idx]["frame"])
            x = int(float(df.iloc[idx]["x"]) * ratio)
            y = int(float(df.iloc[idx]["y"]) * ratio)
            #lbl = df.iloc[idx]["type"]
            print(frame_idx,x,y)
            
            if 0 == list(frames.keys()).count(frame_idx):
                frames[frame_idx] = [(x,y)]
            else:
                frames[frame_idx].append((x,y))
                
    return frames


In [63]:
# objeleri framelerle eşleştirererk birleştiriyoruz
frames = get_labels(objects_labels_files_list)
clear_output()
print(list(frames.keys())[0:3])
print(list(frames.values())[0:3])

[104, 105, 106]
[[(1086, 867), (966, 917), (948, 274), (896, 812), (1042, 220), (1005, 249), (975, 128), (1025, 884), (1092, 220), (1032, 681)], [(1088, 867), (966, 916), (949, 277), (897, 811), (1043, 223), (1006, 252), (975, 131), (1025, 882), (1093, 223), (1032, 678)], [(1088, 864), (969, 916), (947, 278), (898, 809), (1043, 227), (1006, 253), (975, 133), (1025, 882), (1093, 225), (1031, 675)]]


In [64]:
def to_yolo(objects,frame_shape,label = 1,bbox_inner_padding=16):
    width = frame_shape[1]
    height = frame_shape[0]
    
    norm_objects = []
    
    for obj in objects:
        norm_x = obj[0] / width
        norm_y = obj[1] / height
        norm_width = bbox_inner_padding*2 / width
        norm_height = bbox_inner_padding*2 / height
        norm_objects.append(f"{label} {norm_x} {norm_y} {norm_width} {norm_height}\n")
    
    return norm_objects

### videoda göster


In [65]:
cap = cv2.VideoCapture(vid_file_path)
screen_width, screen_height = get_screen_resolution()

frame_counter = 0
rec_inner_padding = 16

while True:
    ret,frame = cap.read()
    if ret:
        frame_ = frame.copy()
        
        try:
            objects_ = frames[frame_counter]
            for obj in objects_:
                cv2.rectangle(frame_,(obj[0]-rec_inner_padding,obj[1]-rec_inner_padding),(obj[0]+rec_inner_padding,obj[1]+rec_inner_padding),(0,0,0),1)
        except:
            pass
        
        cv2.putText(frame_,str(frame_counter),(30,60),cv2.FONT_HERSHEY_DUPLEX,1.8,(0,0,0),)
        
        key = cv2.waitKey(30)
        cv2.namedWindow(vid_file_path)
        frame_ = cv2.resize(frame_,(int(screen_width*0.6),int(screen_height*0.6)))
        cv2.imshow(vid_file_path,frame_)
        cv2.moveWindow(vid_file_path,20,20)
        if key == ord("q"):
            break
        frame_counter += 1
    else:
        break
cv2.destroyAllWindows()
cap.release()

### Frame ve tagleri ayıkla

In [66]:
cap = cv2.VideoCapture(vid_file_path)

vid_name = vid_file_path.split(".")[1][1:]
frame_counter = 0
rec_inner_padding = 16

while True:
    ret,frame = cap.read()
    if ret:
        
        try:
            objects_ = frames[frame_counter]
            print("b")
            yolo_objects = to_yolo(objects_,frame_shape=frame.shape)
            print(len(yolo_objects))
            cv2.imwrite(path_sequence(save_frames_path,f"VCI_{vid_name}_{frame_counter}.jpg"),frame)
            f = open(path_sequence(save_labels_path,f"VCI_{vid_name}_{frame_counter}.txt"),"w")
            f.writelines(yolo_objects)
            f.close()
            print("Yazıldı", frame_counter)
        except:
            pass

        frame_counter += 1
    else:
        break
cv2.destroyAllWindows()
cap.release()

b
10
Yazıldı 104
b
10
Yazıldı 105
b
10
Yazıldı 106
b
10
Yazıldı 107
b
10
Yazıldı 108
b
10
Yazıldı 109
b
10
Yazıldı 110
b
10
Yazıldı 111
b
10
Yazıldı 112
b
10
Yazıldı 113
b
10
Yazıldı 114
b
10
Yazıldı 115
b
10
Yazıldı 116
b
10
Yazıldı 117
b
10
Yazıldı 118
b
10
Yazıldı 119
b
10
Yazıldı 120
b
10
Yazıldı 121
b
10
Yazıldı 122
b
10
Yazıldı 123
b
10
Yazıldı 124
b
10
Yazıldı 125
b
10
Yazıldı 126
b
10
Yazıldı 127
b
10
Yazıldı 128
b
10
Yazıldı 129
b
10
Yazıldı 130
b
10
Yazıldı 131
b
10
Yazıldı 132
b
10
Yazıldı 133
b
10
Yazıldı 134
b
10
Yazıldı 135
b
10
Yazıldı 136
b
10
Yazıldı 137
b
10
Yazıldı 138
b
10
Yazıldı 139
b
10
Yazıldı 140
b
10
Yazıldı 141
b
10
Yazıldı 142
b
10
Yazıldı 143
b
10
Yazıldı 144
b
10
Yazıldı 145
b
10
Yazıldı 146
b
10
Yazıldı 147
b
10
Yazıldı 148
b
10
Yazıldı 149
b
10
Yazıldı 150
b
10
Yazıldı 151
b
10
Yazıldı 152
b
10
Yazıldı 153
b
10
Yazıldı 154
b
10
Yazıldı 155
b
10
Yazıldı 156
b
10
Yazıldı 157
b
10
Yazıldı 158
b
10
Yazıldı 159
b
10
Yazıldı 160
b
10
Yazıldı 161
b
10
Yazıldı 1